# Проект для «Викишоп»

# Содержание

- [Введение](#intro)
- [Подготовка](#preparing)
  - [Лемматизация](#lemma)
  - [Выводы по подготовке](#prep_conclusion)
- [Обучение](#fit)
  - [Логистическая регрессия](#log_reg)
  - [Древо решений](#dt)
  - [Тестовая выборка](#test)
- [Вывод](#conclusion)

<a id='intro'></a>
# Введение

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нам предоставили данные с комментариями пользователей интернет-магазина. В них присутствует разметка о токсичности комментария. Используя эти данные нам необходимо создать модель которая сможет классифицировать комментария на позитивные и негативные. Модель будет считаться успешной если мы получим метрику качества F1 больше 0.75 на тестовых данных.

Прочитаем данные, изучим их и подготовим для использования с моделями машинного обучения. Так как предоставлен только один файл, нам нужно будет разбить данные на выборки для обучения и тестирования. Обучим несколько моделей, сравним их на тренировочных данных, выберем лучшую и посмотрим как она справится с тестовой выборкой.

<a id='preparing'></a>
## Подготовка

**Импортируем необходимые библиотеки**

In [1]:
import os
import pandas as pd
import numpy as np
import warnings

import nltk
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re


from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

**Читаем файл с предоставленными данными**

In [2]:
pth1 = 'toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

**Основная информация о датафрейме**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


В данных находится 159292 строки (коментарии). Присутсвует колонка `Unnamed: 0` которая нам не понадобится, можем её удалять. Но сначала убедимся что дубликаты и пропущеные значения отсутсвуют.

**Дубликаты, пропуски**

In [5]:
data.duplicated().sum()

0

In [6]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Дубликаты и пропуски отсутсвуют.

**Удаление лишних данных**

In [7]:
data = data.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Колонку удалили, теперь в нашем распоряжении две колонки - одна с текстом комментария, вторая с маркером токсичности.

Посмотрим на баланс классов.

In [9]:
frequency = data['toxic'].value_counts(normalize=True)
print(frequency)

0    0.898388
1    0.101612
Name: toxic, dtype: float64


Присутсвует небаланс классов. 90% - не токсичные комментарии и 10% - токсичные.

<a id='lemma'></a>
### Лемматизация

Нам нужно обработать текст прежде чем обучать модели. Напишем фукцию для очитки текста от лишних символов, приведём текст к одному регистру, а так же приведём слова к их изначальной форме. Для лемматизации будем использоваить библиотеку Mystem.

In [10]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized_words)

In [12]:
%%time

data['text'] = data['text'].apply(preprocess_text)

CPU times: user 7min 1s, sys: 2.25 s, total: 7min 4s
Wall time: 7min 4s


In [13]:
data.head(10)

,text,toxic
0,explanation why the edits make under my userna...,0
1,daww he match this background colour im seemin...,0
2,hey man im really not try to edit war it just ...,0
3,more i cant make any real suggestion on improv...,0
4,you sir be my hero any chance you remember wha...,0
5,congratulation from me a well use the tool wel...,0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word nonsense be offensive to you...,0
9,alignment on this subject and which be contrar...,0


Обработку теста завершили, приступим к получению выборок. Разобьём данные в соотношении 80%/20% - тренировочная и тестовая выборка соответсвенно.

In [14]:
RANDOM_STATE = 12345

In [15]:
features = data['text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.2,
                                                                            random_state=RANDOM_STATE)

<a id='prep_conclusion'></a>
### Выводы по подготовке

Изучили предоставленные данные. Пропусков и дубликатов в них нет. Удалили лишний столбец который предположительно содержал индексы комментарий. Произвели очистку текста и лемматизацию. После этого сделали разбику на две выборки, тренировочную и тестовую в соотношении 80%/20%. Данные готовы для работы с моделями машинного обучения.

<a id='fit'></a>
## Обучение

In [17]:
folds = KFold(n_splits = 5, shuffle = True, random_state = RANDOM_STATE)

In [18]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id='log_reg'></a>
### Логистическая регрессия

Подберем параметры для логистической регрессии с помощью кросс-валидации GridSearchCV. Так же будем делать TF-IDF векторизацию, пропишем её в pipeline.

In [19]:
pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words=stopwords)),
    ('lr', LogisticRegression(random_state=RANDOM_STATE, max_iter=2000))
])

In [20]:
%%time

param_grid_lr = {
                 'lr__class_weight': [None, 'balanced'],
                 'lr__solver': ['lbfgs', 'liblinear'],
                 'lr__C': [0.1, 1, 10]
                }

grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, cv=folds, scoring='f1')

grid_search_lr.fit(features_train, target_train)

best_params_lr = grid_search_lr.best_params_
best_f1_lr = grid_search_lr.best_score_
best_lr_model = grid_search_lr.best_estimator_

print("Best Parameters:", best_params_lr)
print("Best F1 on training data:", best_f1_lr)

Best Parameters: {'lr__C': 10, 'lr__class_weight': None, 'lr__solver': 'lbfgs'}
Best F1 on training data: 0.7611707246445614
CPU times: user 6min 4s, sys: 2.14 s, total: 6min 7s
Wall time: 6min 7s


Параметры подобрали и сохранили лучшую модель. На тренировочных данных получили **F1 = 0.761**.

<a id='dt'></a>
### Древо решений

Подберем параметры для древа решений с помощью кросс-валидации GridSearchCV. Так же будем делать TF-IDF векторизацию, пропишем её в pipeline.

In [22]:
pipeline_dt = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words=stopwords)),
    ('dt', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [23]:
%%time

param_grid_dt = {
                 'dt__class_weight': [None, 'balanced'],
                }

grid_search_dt = GridSearchCV(pipeline_dt, param_grid_dt, cv=folds, scoring='f1')

grid_search_dt.fit(features_train, target_train)

best_params_dt = grid_search_dt.best_params_
best_f1_dt = grid_search_dt.best_score_

print("Best Parameters for Decision Tree:", best_params_dt)
print("Best F1 for Decision Tree:", best_f1_dt)

Best Parameters for Decision Tree: {'dt__class_weight': None}
Best F1 for Decision Tree: 0.6850131015995525
CPU times: user 19min 28s, sys: 880 ms, total: 19min 29s
Wall time: 19min 29s


Параметры подобрали и сохранили лучшую модель. На тренировочных данных получили **F1 = 0.685**.

<a id='test'></a>
### Тестовая выборка

Обучили 2 модели, из них лучший результат на тренировочных данных показала логистическая регрессия. Посмотрим как она справится с тестовой выборкой.

In [25]:
predictions = best_lr_model.predict(features_test)
f1_test = f1_score(target_test, predictions)
print('F1 on training data: ', f1_test)

F1 on training data:  0.7659426585577759


Получили **F1 = 0.766** на тестовой выборке, данных результат нас устраивает.

<a id='conclusion'></a>
## Выводы

In [26]:
chart_data = [
              ["Логистическа регрессия", best_f1_lr, f1_test],
              ["Древо решений", best_f1_dt, "-"],
]

headers = ["Модель", "F1 на тренировочный данных", "F1 на тестовых данных"]

chart = pd.DataFrame(chart_data, columns=headers)

display(chart)

,Модель,F1 на тренировочный данных,F1 на тестовых данных
0,Логистическа регрессия,0.761171,0.765943
1,Древо решений,0.685013,-


Нам необходимо было создать модель машинного обучения которая сможет классифицировать комментарии пользователей интернет-магазина на токсичные и нет опираясь на метрику качества F1 на тестовой выборке. Нам удалось достич этой цели, мы получили результат **F1 = 0.766** на тестовой выборке. В качестве модели использовалась логистическая регрессия.

Перед этим мы изучили предоставленные нам данные. Из ним мы удалили не нужную нам колонку содержащую предположительно id сообщения. В дальнейшем нам необходимо было выполнить очистку текста и лемматизацию. Использовалась библиотека WordNetLemmatizer для этих целей, а также stopwords из nltk. Получив обработанный текст мы произвели разделение данных на выборки в соотнишении 80%/20% - тренировочная и тестовая соответсвенно. Одна нам необходима для обучения моделей, а вторая для конечного тестирования. Подбор параметров мы выполняли с помощью кросс-валидации GridSearchCV. Сравнивали две модели Логистическая регрессия и древо решейний. Первая показала себя намного лучше, вторая даже на тренировочных данных не смогла достичнуть минимально необходимой метрики и к тому же она намного медленнее обучалась.

В общем остановились на Логистической регрессии, её результаты нас устраивают.